# Cleanup All Resources

Each notebook has a resource cleanup section that is designed to remove just the resources it created.

This notebook is designed to remove all resources by the type created.  By being a general cleanup utility, you can skip sections that you want to remain in use.  This notebook also has a section to delete the GCS bucket created in the `00 - Initial Setup` notebook.

**Resources**
- Vertex AI: https://googleapis.dev/python/aiplatform/latest/index.html
- BigQuery: https://googleapis.dev/python/bigquery/latest/index.html
- GCS: https://googleapis.dev/python/storage/latest/client.html

---
## Setup

Parameters:

In [10]:
PROJECT_ID = 'statmike-mlops'
REGION = 'us-central1'

PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

AI Platform Client Setup:

In [11]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1beta1

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

aiplatform.init(project=PROJECT_ID, location=REGION)

PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

---
## BigQuery
- Delete the dataset `digits`

Make a client connection to BigQuery

In [12]:
from google.cloud import bigquery
clients['bq'] = bigquery.Client(project=PROJECT_ID)

list of datasets in the project:

In [13]:
datasets = list(clients['bq'].list_datasets())

delete each dataset:

In [14]:
for ds in datasets:
    print(ds.dataset_id)
    clients['bq'].delete_dataset(ds.dataset_id,delete_contents=True,not_found_ok=True)

digits
model_deployment_monitoring_7199267887142404096


Forbidden: 403 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/statmike-mlops/datasets/model_deployment_monitoring_7199267887142404096?deleteContents=true&prettyPrint=false: Access Denied: Dataset statmike-mlops:model_deployment_monitoring_7199267887142404096: Permission bigquery.datasets.delete denied on dataset statmike-mlops:model_deployment_monitoring_7199267887142404096 (or it may not exist).

---
## Vertex AI > Datasets
- Delete datasets

In [15]:
datasets = aiplatform.TabularDataset.list()

In [16]:
for ds in datasets:
    print(ds.resource_name)
    aiplatform.TabularDataset(dataset_name=ds.resource_name).delete()

projects/691911073727/locations/us-central1/datasets/5098945591392600064
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/5098945591392600064
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/11244430539358208
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/5098945591392600064
projects/691911073727/locations/us-central1/datasets/7530889390172667904
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/7530889390172667904
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/4622930448966746112
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/7530889390172667904
proj

---
## Vertex AI > Features
- delete feature store including entity types and features

In [16]:
clients['fs'] = aiplatform_v1beta1.FeaturestoreServiceClient(client_options=client_options)

In [17]:
fs = clients['fs'].list_featurestores(parent=PARENT)

In [18]:
for f in fs:
    print(f.name)
    clients['fs'].delete_featurestore(request = aiplatform_v1beta1.types.DeleteFeaturestoreRequest(name=f.name,force=True))

projects/691911073727/locations/us-central1/featurestores/digits_featurestore


---
## Vertex AI > Pipelines
* Delete Managed Pipelines

In [17]:
pl = aiplatform.PipelineJob.list()

In [18]:
for p in pl:
    print(p.name)
    p.delete()

kfp-02c-digits-20210915141811-20210915142043
INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210915141811-20210915142043
INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/691911073727/locations/us-central1/operations/2920569789820698624
INFO:google.cloud.aiplatform.base:PipelineJob deleted. . Resource name: projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210915141811-20210915142043
kfp-02c-digits-20210915135548
INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210915135548
INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/691911073727/locations/us-central1/operations/1524453905335844864
INFO:google.cloud.aiplatform.base:PipelineJob deleted. . Resource name: projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210915135548


---
## Vertex AI > Training
- delete various jobs types
- some create with aiplatform.* and other created by aiplatform.gapic.*

## Vertex AI > Training > Training Pipelines
- Delete training pipelines

In [19]:
jobs = aiplatform.CustomTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

In [20]:
jobs = aiplatform.CustomPythonPackageTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

In [21]:
jobs = aiplatform.AutoMLTabularTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

02c_digits_20210915141811
INFO:google.cloud.aiplatform.base:Deleting AutoMLTabularTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/6531118860198739968
INFO:google.cloud.aiplatform.base:Delete AutoMLTabularTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/6339927806901747712
INFO:google.cloud.aiplatform.base:AutoMLTabularTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/6531118860198739968
02c_digits
INFO:google.cloud.aiplatform.base:Deleting AutoMLTabularTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/7328255994243317760
INFO:google.cloud.aiplatform.base:Delete AutoMLTabularTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/8685177312854933504
INFO:google.cloud.aiplatform.base:AutoMLTabularTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/7328255994243317760
02b_digits_2021

## Vertex AI > Training > Custom Job
- Delete custom training jobs

In [22]:
jobs = aiplatform.CustomJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

caip_pipelines_288450278477266944_3207064311990059008
INFO:google.cloud.aiplatform.base:Deleting CustomJob : projects/691911073727/locations/us-central1/customJobs/2083814228170375168
INFO:google.cloud.aiplatform.base:Delete CustomJob  backing LRO: projects/691911073727/locations/us-central1/operations/7865522180673503232
INFO:google.cloud.aiplatform.base:CustomJob deleted. . Resource name: projects/691911073727/locations/us-central1/customJobs/2083814228170375168
caip_pipelines_288450278477266944_-3710464715651022848
INFO:google.cloud.aiplatform.base:Deleting CustomJob : projects/691911073727/locations/us-central1/customJobs/7593968372258177024
INFO:google.cloud.aiplatform.base:Delete CustomJob  backing LRO: projects/691911073727/locations/us-central1/operations/4469808061636149248
INFO:google.cloud.aiplatform.base:CustomJob deleted. . Resource name: projects/691911073727/locations/us-central1/customJobs/7593968372258177024
caip_pipelines_288450278477266944_-6016307724864716800
INFO:g

## Vertex AI > Training > Hyperparameter Tuning Job

In [23]:
jobs = aiplatform.HyperparameterTuningJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

---
## Vertex AI > Experiments > Experiment & Tensorboard Instances
ISSUE: remove experiments first then instances.

In [24]:
tb = aiplatform.Tensorboard.list()
for t in tb:
    print(t)
    t.delete()

resource name: projects/691911073727/locations/us-central1/tensorboards/2913723405792444416
INFO:google.cloud.aiplatform.base:Deleting Tensorboard : projects/691911073727/locations/us-central1/tensorboards/2913723405792444416
INFO:google.cloud.aiplatform.base:Delete Tensorboard  backing LRO: projects/691911073727/locations/us-central1/operations/2893548192056475648
INFO:google.cloud.aiplatform.base:Tensorboard deleted. . Resource name: projects/691911073727/locations/us-central1/tensorboards/2913723405792444416


In [44]:
clients['tb'] = aiplatform_v1beta1.TensorboardServiceClient(client_options=client_options)

In [45]:
tbs = clients['tb'].list_tensorboards(parent=PARENT)

In [46]:
for tb in tbs:
    print(tb.name)
    tbes = clients['tb'].list_tensorboard_experiments(parent=tb.name)
    for tbe in tbes:
        print(tbe.name)
        clients['tb'].delete_tensorboard_experiment(request=aiplatform_v1beta1.types.DeleteTensorboardExperimentRequest(name=tbe.name))
    clients['tb'].delete_tensorboard(request = aiplatform_v1beta1.types.DeleteTensorboardRequest(name=tb.name))

## Vertex AI > Experiments > Studies
Vizier Studies

In [27]:
clients['viz'] = aiplatform_v1beta1.VizierServiceClient(client_options=client_options)

In [28]:
studies = clients['viz'].list_studies(parent=PARENT)

In [29]:
for study in studies:
    print(study.name)
    clients['viz'].delete_study(name=study.name)

projects/691911073727/locations/us-central1/studies/4198436052403
projects/691911073727/locations/us-central1/studies/2954684456476


---
## Vertex AI > Endpoints
- Delete endpoints forcing deployed models to undeploy

In [25]:
endpoints = aiplatform.Endpoint.list()

In [32]:
for ep in endpoints:
    print(ep.resource_name)
    ep.delete(force=True)

projects/691911073727/locations/us-central1/endpoints/5021777467308769280
INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/691911073727/locations/us-central1/endpoints/5021777467308769280
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/5021777467308769280/operations/299474806691069952
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/691911073727/locations/us-central1/endpoints/5021777467308769280
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/691911073727/locations/us-central1/endpoints/5021777467308769280
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/691911073727/locations/us-central1/operations/5992024735687376896
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/691911073727/locations/us-central1/endpoints/5021777467308769280
projects/691911073727/locations/us-central1/endpo

---
## Vertex AI > Models
- Delete endpoints first (contain deployed models)
- Delete uploaded models

In [33]:
models = aiplatform.Model.list()

In [34]:
for model in models:
    print(model.resource_name)
    model.delete()

projects/691911073727/locations/us-central1/models/6536614219314364416
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/6536614219314364416
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/7820486184399798272
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/6536614219314364416
projects/691911073727/locations/us-central1/models/672927504477978624
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/672927504477978624
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/4433779264617185280
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/672927504477978624
projects/691911073727/locations/us-central1/models/1466686938802028544


---
## Vertex AI > Batch Predictions
- delete all batch prediction jobs

In [35]:
bps = aiplatform.BatchPredictionJob.list()

In [36]:
for bp in bps:
    print(bp.resource_name)
    bp.delete()

projects/691911073727/locations/us-central1/batchPredictionJobs/7411572587349671936
INFO:google.cloud.aiplatform.base:Deleting BatchPredictionJob : projects/691911073727/locations/us-central1/batchPredictionJobs/7411572587349671936
INFO:google.cloud.aiplatform.base:Delete BatchPredictionJob  backing LRO: projects/691911073727/locations/us-central1/operations/4613923249712005120
INFO:google.cloud.aiplatform.base:BatchPredictionJob deleted. . Resource name: projects/691911073727/locations/us-central1/batchPredictionJobs/7411572587349671936
projects/691911073727/locations/us-central1/batchPredictionJobs/3329903950068449280
INFO:google.cloud.aiplatform.base:Deleting BatchPredictionJob : projects/691911073727/locations/us-central1/batchPredictionJobs/3329903950068449280
INFO:google.cloud.aiplatform.base:Delete BatchPredictionJob  backing LRO: projects/691911073727/locations/us-central1/operations/3758239320511610880
INFO:google.cloud.aiplatform.base:BatchPredictionJob deleted. . Resource na

---
## Vertex AI > Metadata

In [37]:
clients['md'] = aiplatform_v1beta1.MetadataServiceClient(client_options=client_options)

In [38]:
md = clients['md'].list_metadata_stores(parent=PARENT)

In [39]:
for m in md:
    print(m.name)
    arts = clients['md'].list_artifacts(parent=m.name)
    for a in arts:
        print(a)
        clients['md'].delete_artifact(name=a.name)
    #clients['md'].get_metadata_store(name=m.name)#.purge_artifacts(parent=m.name)

projects/691911073727/locations/us-central1/metadataStores/default
name: "projects/691911073727/locations/us-central1/metadataStores/default/artifacts/11435714960211677443"
display_name: "endpoint"
uri: "aiplatform://v1/projects/691911073727/locations/us-central1/endpoints/3537841390090190848"
etag: "1631721239211"
create_time {
  seconds: 1631720313
  nanos: 984000000
}
update_time {
  seconds: 1631721239
  nanos: 211000000
}
state: LIVE
schema_title: "system.Artifact"
schema_version: "0.0.1"
metadata {
}

name: "projects/691911073727/locations/us-central1/metadataStores/default/artifacts/17120282799001191112"
display_name: "model"
uri: "aiplatform://v1/projects/691911073727/locations/us-central1/models/672927504477978624"
etag: "1631720312791"
create_time {
  seconds: 1631715765
  nanos: 271000000
}
update_time {
  seconds: 1631720312
  nanos: 791000000
}
state: LIVE
schema_title: "system.Model"
schema_version: "0.0.1"
metadata {
}

name: "projects/691911073727/locations/us-central1/

---
## Storage Bucket Removal

- delete contents and storage bucket `statmike-mlops` (named after project)

In [40]:
from google.cloud import storage
clients['gcs'] = storage.Client()

In [41]:
bucket = clients['gcs'].get_bucket(PROJECT_ID)

In [42]:
bucket.delete(force=True)